# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
collection_list = db.list_collection_names()
print(collection_list)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {
    "scores.Hygiene": 20
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
result = establishments.find_one(query)
pprint(result)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

# Display the first document in the results using pprint
result = establishments.find_one(query)
pprint(result)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the number of rows in the DataFrame
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
# Get the latitude and longitude of "Penang Flavours" restaurant
penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})
latitude = penang_flavours["geocode"]["latitude"]
longitude = penang_flavours["geocode"]["longitude"]

# Define the degree search radius
degree_search = 0.01

# Define the query to find establishments within the search radius with RatingValue equal to 5
query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

# Define the sort order by hygiene score
sort = [("scores.Hygiene", 1)]

# Find the top 5 establishments matching the query and sort order
result = establishments.find(query).sort(sort).limit(5)

# Print the results
for establishment in result:
    pprint(establishment)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(result))

# Print the DataFrame
print(df)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create an aggregation pipeline
# Matches establishments with a hygiene score of 0
# Groups the matches by Local Authority
# Sorts the matches from highest to lowest
pipeline = [
    {
        "$match": {
            "scores.Hygiene": 0
        }
    },
    {
        "$group": {
            "_id": "$LocalAuthorityName",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    }
]

# Execute the aggregation pipeline and store the result in a variable
result = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print("Number of documents with a hygiene score of 0 in each Local Authority area:")

# Print the first 10 results
for item in result[:10]:
    print(f"{item['_id']}: {item['count']} establishments")

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))